# Python Astro Photo processing

Welcome to this project about processing astro photos with Python. This project aims to correct errors in photographs taken near a source of light pollution, such as a city, and to improve the quality of the image.

In [43]:
import cv2
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [38]:
# Chargement de l'image

INPUT_FILE = 'img/input/barnard_stacked_gradient.png'

img = cv2.imread(INPUT_FILE)

In [ ]:
# Method 1 : Value threshold (HSV)

# Advantages: Easy to implement, fast, works well in most cases
# Disadvantages: Can remove bright pixels in the image

OUTPUT_FILE = 'img/output/barnard_stacked_gradient_threshold.png'

# ----------------------------

mask = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
# Filter out brightest pixels
mask = cv2.inRange(mask, (0, 0, 0), (255, 255, 200))

# Substract mask from image
mask = cv2.bitwise_not(mask)
res = cv2.bitwise_and(img, img, mask=mask)


# Compare original and result
plt.imshow(np.hstack((img, res)))


# save img to output folder
cv2.imwrite('img/output/barnard_stacked_gradient_value_clip.png', res)

In [ ]:
# Method 2 : Edge detection surface removal

# Advantages: 
# Disadvantages:

# ----------------------------



In [59]:
# Method 3 : Manual point input (interpolation)

# Advantages: Quick
# Disadvantages: Not accurate, manual input

OUTPUT_FILE = 'img/output/barnard_stacked_gradient_manual.png'
matplotlib.use('TkAgg')

# ----------------------------


plt.imshow(img)
plt.show()
points = plt.ginput(4)
# print(points)
plt.close()

# Get color of each point
colors = []
for point in points:
    colors.append(img[int(point[1]), int(point[0])])

# Interpolate for whole image
res = img.copy()
for i in range(img.shape[0]):
    for j in range(img.shape[1]):
        for k in range(3):
            res[i, j, k] = np.interp(img[i, j, k], [colors[0][k], colors[1][k]], [0, 255])

# Convert res colorspace
res = cv2.cvtColor(res, cv2.COLOR_BGR2HSV)


# save img to output folder
cv2.imwrite(OUTPUT_FILE, res)

ValueError: object too deep for desired array